<a href="https://colab.research.google.com/github/evegat/usm2025-1-dip-ia-edu-v2-evegat/blob/main/InformeFinalDIPLOIAUSMEVT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dimensiones contextuales que promueven el interés por estudiar pedagogía en los egresados de cuarto medio  
## Detector de intereses para la Región Metropolitana  

---

## 1. Introducción

Chile enfrenta una crisis sostenida en la formación de nuevos docentes, evidenciada por una disminución en la matrícula de carreras de pedagogía, particularmente en regiones. Frente a ello, resulta urgente desarrollar herramientas que permitan identificar tempranamente los perfiles con interés o probabilidad de seguir estas carreras, para orientar de forma más eficaz las políticas públicas.

Este trabajo propone el desarrollo de un modelo de clasificación binaria mediante Machine Learning que prediga si una persona estudiará pedagogía o no, en base a variables demográficas, académicas y contextuales extraídas de bases públicas del MINEDUC. La solución está pensada como un **detector de vocación docente temprana**, con utilidad para la asignación de recursos, el diseño de campañas de atracción y la focalización territorial de esfuerzos institucionales.

---

## 2. Objetivo del modelo

**Objetivo general:** Construir un modelo supervisado de clasificación para estimar si un estudiante de primer año optará por pedagogía, a partir de datos consolidados de matrícula.

**Objetivos específicos:**
- Implementar y comparar modelos supervisados que permitan realizar esta clasificación.
- Identificar variables predictivas relevantes asociadas a la elección de pedagogía.
- Evaluar su potencial aplicación en estrategias de política pública a nivel regional.

---

## 3. Descripción del Conjunto de Datos

### Fuente:
Portal de Datos Abiertos del Ministerio de Educación:  
👉 https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

### Contenido:
La base contiene más de 1.5 millones de registros de matrícula en educación superior entre 2023 y 2024. Se incluyen variables como región, jornada, forma de ingreso, tipo de institución, acreditación y nombre de carrera.

### Filtrado:
Para el problema de interés se consideró exclusivamente la matrícula **a primer año**, por ser el momento en que se realiza la elección de carrera inicial. Dado el volumen de datos (900 MB), se generó una muestra aleatoria representativa para asegurar manejo técnico viable sin pérdida de diversidad.

---

## 4. Distribución y Preprocesamiento

Este proceso fue documentado en los notebooks `03analisis.ipynb` y `04preprocesamiento.ipynb`.

### a. Análisis exploratorio:
Se analizaron tipos de datos, nulos, valores extremos y relaciones entre variables:
- `valor_matricula` y `valor_arancel` mostraron asimetría y fueron normalizados.
- La variable `nomb_carrera` permitió derivar el campo `es_pedagogia` (variable objetivo).

### b. Eliminación de duplicados:
Más del 70% de los registros resultaron duplicados. Se eliminaron 1.2 millones de filas con `df.drop_duplicates()`.

### c. Manejo de valores nulos:
Solo 20 filas tenían nulos en arancel o matrícula. Se optó por `df.dropna()` para evitar introducir sesgo o ruido con imputaciones.

### d. Codificación de variables categóricas:
Se utilizó `LabelEncoder` para transformar variables como `region_sede`, `jornada`, `modalidad`, etc. El procedimiento fue aplicado columna por columna, garantizando consistencia con el entrenamiento.

### e. Normalización:
Se aplicó `MinMaxScaler()` a `valor_matricula` y `valor_arancel` por tener rangos muy amplios y alta dispersión. Esto previene que los modelos sesguen su decisión hacia variables de gran magnitud.

---

## 5. Métodos y Modelos Utilizados

Notebook: `05entrenamiento.ipynb`

### Modelos seleccionados:

| Modelo | Justificación |
|--------|---------------|
| **DecisionTreeClassifier** | Elegido por su interpretabilidad y facilidad para visualizar reglas de decisión. Útil para entender qué variables tienen más peso. Ideal como línea base. |
| **RandomForestClassifier** | Basado en múltiples árboles, reduce el sobreajuste y mejora la capacidad predictiva. Es robusto ante ruido, escalable y adecuado para conjuntos con mezcla de variables categóricas y numéricas. |

### Hiperparámetros y razones:

| Modelo        | Hiperparámetros            | Justificación |
|---------------|----------------------------|----------------|
| Árbol de decisión | `max_depth=4`             | Controla el crecimiento excesivo del árbol, evita sobreajuste y mejora la legibilidad del modelo. |
| Random Forest | `n_estimators=100`<br>`max_depth=10`<br>`random_state=42` | 100 árboles ofrecen buen balance entre precisión y rendimiento. Se fija una profundidad moderada para evitar sobreajuste. Se fija una semilla para reproducibilidad. |

### Partición de datos:
Se usó `train_test_split(test_size=0.2, stratify=y)` para asegurar distribución balanceada de la clase minoritaria (`es_pedagogia=1`). Esto previene sesgos y asegura comparabilidad.

---

## 6. Evaluación del Rendimiento del Modelo

Notebook: `06validacion.ipynb`

### Métricas utilizadas:
- **Accuracy:** proporción de clasificaciones correctas.
- **Precision:** minimiza falsos positivos (útil si queremos evitar sobreestimaciones de vocación).
- **Recall:** minimiza falsos negativos (importante para no perder casos reales con interés en pedagogía).
- **F1-score:** balance entre precisión y recall.
- **Matriz de confusión:** para observar aciertos por clase.

### Resultados:

| Modelo              | Accuracy | Precision (1) | Recall (1) | F1-score |
|---------------------|----------|----------------|------------|----------|
| Árbol de Decisión   | 95.8%    | 0.76           | 0.75       | 0.76     |
| **Random Forest**   | **96.9%**| **0.85**       | **0.77**   | **0.81** |

**Conclusión crítica:**  
El Árbol de Decisión es más interpretable, pero Random Forest entrega mejores resultados sin pérdida significativa de explicabilidad. El rendimiento es sólido en ambas clases, pero destaca su capacidad para recuperar casos positivos sin incurrir en sobreajuste, lo que se evidencia al comparar el accuracy en entrenamiento vs testeo (ambos en torno al 97%).

---

## 7. Interpretación de Resultados

- Variables como `tipo_inst_1`, `region_sede_cod`, `forma_ingreso`, `jornada` y `nivel_carrera_1` fueron recurrentemente seleccionadas como importantes.
- El modelo detecta que los perfiles más probables a elegir pedagogía se asocian a instituciones técnicas acreditadas, modalidad presencial, jornada diurna y formas de ingreso como PACE.
- La matriz de confusión muestra menor cantidad de falsos negativos en Random Forest, lo cual es clave en políticas públicas: **mejor captar a quienes sí tienen vocación que subestimar su presencia.**

---

## 8. Prueba con ejemplos reales

Notebook: `07prueba.ipynb`

Se construyeron tres ejemplos sintéticos con características realistas:

```plaintext
Ejemplo 1: Mujer, joven, institución técnica acreditada, jornada diurna, presencial → No pedagogía  
Ejemplo 2: Hombre, mayor, institución profesional no acreditada, online → No pedagogía  
Ejemplo 3: Hombre, joven, jornada diurna, con arancel alto, forma ingreso especial → No pedagogía
```

Aunque ninguno fue clasificado como "pedagogía", esto es consistente con los patrones detectados por el modelo: la forma de ingreso, acreditación y tipo de institución son decisivos.

---

## 9. Conclusiones

Se logró construir un modelo predictivo robusto, con buen rendimiento y aplicabilidad inmediata en planificación institucional.

La integración de fuentes públicas, un pipeline reproducible y métricas claras hacen del modelo una herramienta transparente y replicable.

Puede ser implementado como insumo para estrategias focalizadas en regiones con baja matrícula pedagógica.

Recomendamos avanzar hacia una arquitectura con más variables (socioeconómicas, rendimiento escolar) y explorar modelos como XGBoost o redes neuronales con explainability integrada.

---

## 10. Referencias

Ministerio de Educación de Chile. (2024). Matrícula en Educación Superior.  
https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/
